In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import warnings
warnings.filterwarnings("ignore")

In [2]:
#Import keras functions

import tensorflow as tf
print(tf.__version__)

from keras import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD,Adam
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications import VGG19,Xception
from tensorflow.keras.layers import Input,Flatten,Dense,BatchNormalization,Activation,Dropout,GlobalAveragePooling2D,MaxPooling2D,RandomFlip,RandomZoom,RandomRotation


2.15.0


In [3]:
#Import the dataset

from keras.datasets import cifar10

In [4]:
(x_train, y_train), (x_val, y_val) = cifar10.load_data()

# xception.preprocess_input(): the inputs pixel values are scaled between -1 and 1
#x_train = tf.keras.applications.xception.preprocess_input(x_train)
#x_val = tf.keras.applications.xception.preprocess_input(x_val)

y_train=to_categorical(y_train)
y_val=to_categorical(y_val)

print((x_train.shape, y_train.shape))
print((x_val.shape, y_val.shape))


((50000, 32, 32, 3), (50000, 10))
((10000, 32, 32, 3), (10000, 10))


In [5]:
base_model = Xception(include_top=False, weights='imagenet', input_shape=(224,224,3), classes=y_train.shape[1])

In [6]:
# freeze all layers
base_model.trainable = False

# to freeze only some of the layers
# index 0 is the input layer
#for layer in base_model.layers[:12]:
#    layer.trainable = False

In [7]:
data_augmentation = Sequential(
    [RandomFlip("horizontal"), 
     RandomRotation(0.1),
     RandomZoom(0.1)]
)

In [8]:
inputs = tf.keras.Input(shape=(32, 32, 3))
x = tf.keras.layers.Lambda(lambda image: tf.image.resize(image, (224,224)))(inputs)
x = data_augmentation(x)
x = tf.keras.applications.xception.preprocess_input(x)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.3)(x)
outputs = tf.keras.layers.Dense(10, activation=('softmax'))(x)
model = tf.keras.Model(inputs, outputs)

In [9]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 lambda (Lambda)             (None, 224, 224, 3)       0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 tf.math.truediv (TFOpLambd  (None, 224, 224, 3)       0         
 a)                                                              
                                                                 
 tf.math.subtract (TFOpLamb  (None, 224, 224, 3)       0         
 da)                                                             
                                                                 
 xception (Functional)       (None, 7, 7, 2048)        208614

In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
epochs = 20
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=epochs, verbose=1)

Epoch 1/20


1563/1563 [==============================] - 1236s 787ms/step - loss: 0.7506 - accuracy: 0.7457 - val_loss: 0.4289 - val_accuracy: 0.8576
Epoch 2/20
1563/1563 [==============================] - 1192s 763ms/step - loss: 0.6375 - accuracy: 0.7799 - val_loss: 0.4154 - val_accuracy: 0.8603
Epoch 3/20
1563/1563 [==============================] - 1192s 763ms/step - loss: 0.6219 - accuracy: 0.7857 - val_loss: 0.3962 - val_accuracy: 0.8643
Epoch 4/20
1563/1563 [==============================] - 1152s 737ms/step - loss: 0.6166 - accuracy: 0.7886 - val_loss: 0.3877 - val_accuracy: 0.8663
Epoch 5/20
1563/1563 [==============================] - 890s 570ms/step - loss: 0.6159 - accuracy: 0.7898 - val_loss: 0.3901 - val_accuracy: 0.8688
Epoch 6/20
1563/1563 [==============================] - 890s 569ms/step - loss: 0.6081 - accuracy: 0.7915 - val_loss: 0.3988 - val_accuracy: 0.8682
Epoch 7/20
1563/1563 [==============================] - 889s 569ms/step - loss: 0.6047 - accuracy: 0.7938 


KeyboardInterrupt



In [ ]:
def plot_history(history):
    history_frame = pd.DataFrame(history.history)
    history_frame.loc[:, ['loss', 'val_loss']].plot()
    history_frame.loc[:, ['accuracy', 'val_accuracy']].plot()
    
    return
    
plot_history(history)

In [ ]:
# unfreeze the layers of the pre-trained model

base_model.trainable = True

In [ ]:
# Use a small learning rate

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
epochs = 10
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=epochs, verbose=1)

In [ ]:
plot_history(history)

In [ ]:
class_names=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

predictions=model.predict(x_val)

y_pred_classes = np.argmax(predictions, axis=1)
y_true = np.argmax(y_val, axis=1)

confusion_mtx = tf.math.confusion_matrix(y_true, y_pred_classes)

plt.figure(figsize=(12, 9))
c = sns.heatmap(confusion_mtx, annot=True, fmt='g')
c.set(xticklabels=class_names, yticklabels=class_names)